In [12]:
import spotipy
import pylast
import pandas as pd
import numpy as np
import json
from datetime import datetime
import calendar
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

## LastFM

In [2]:
lastfm_api = json.load(open("lastfm_api.json", "r"))

In [3]:
API_KEY = lastfm_api['API key']
API_SECRET = lastfm_api['Shared secret']
username = lastfm_api['User']
password_hash = lastfm_api['Password_MD5']

network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET,
                               username=username, password_hash=password_hash)

user = pylast.User(user_name=username, network=network)

In [4]:
network.enable_rate_limit()

In [5]:
dt_from = calendar.timegm(datetime(2017, 12, 1).utctimetuple())
dt_to = calendar.timegm(datetime(2017, 12, 31).utctimetuple())

In [6]:
data = user.get_recent_tracks(limit=None, time_from=dt_from, time_to=dt_to)

In [7]:
def get_track_info(track):
    return({
        "Title": track.title,
        "Artist": track.artist.name,
    })

In [8]:
get_track_info(data[0].track)

{'Title': 'Quem Mando Tu Terminar? (To Bonito To Né?)', 'Artist': 'MC Kekel'}

In [9]:
def get_top_tracks(data, n=70):
    r = []
    for track in data:
        r.append(get_track_info(track.track))
    
    df = pd.DataFrame.from_dict(r)
    df = df.reset_index().groupby(["Title", "Artist"]).count()
    df = df.sort_values("index", ascending=False)
    if (n > 0):
        return df.head(n)
    else:
        return(df)
        

In [10]:
df = get_top_tracks(data)

In [11]:
df.head()

,,index
Title,Artist,
Ooorra... (a que deu nome a mixtape),Emicida,36
Six Days - Remix,DJ Shadow,15
Deixe Me Ir - Acústico,1Kilo,12
Get Away,Big K.R.I.T.,10
Heartbeats,José González,7


## Spotify

In [96]:
spotify_api = json.load(open("spotify_api.json", "r"))

In [97]:
spotify_api

{'Application Name': 'LastFy',
 'Username': 'gasp311',
 'Client ID': 'd234876184c443e6a59df57008eb7b07',
 'Client Secret': '95248e65e2af478dac041b4adc7325a2'}

In [ ]:
spotify = spotipy.Spotify()
token = util.prompt_for_user_token(username, 
                                   scope='playlist-modify-private,playlist-modify-public', 
                                   client_id='enter_client_id', 
                                   client_secret='enter_client_secret', redirect_uri='https://localhost:8080')